<a href="https://colab.research.google.com/github/hrshbond/Language-Translator/blob/main/sanskrit_translator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
%tensorflow_version 1.x
import tensorflow as tf
import keras
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers import Dense
from keras.models import Model

TensorFlow 1.x selected.


Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
%cd ../

/


In [4]:
%ls

bin/      dev/   lib32/  opt/         run/   tensorflow-1.15.2/  var/
boot/     etc/   lib64/  proc/        sbin/  tmp/
content/  home/  media/  python-apt/  srv/   tools/
datalab/  lib/   mnt/    root/        sys/   usr/


In [5]:
%cd /content/drive/My Drive

/content/drive/My Drive


In [7]:
lines= pd.read_csv('/content/drive/My Drive/sans', names=['hin','eng'])

In [8]:
lines = lines.iloc[1:]
lines.index = lines.index.astype(int)


lines.head()

,hin,eng
0,धृतराष्ट्र उवाच |धर्मक्षेत्रे कुरुक्षेत्रे समव...,"Dhritarashtra said: O Sanjay, after gathering ..."
1,सञ्जय उवाच ।दृष्ट्वा तु पाण्डवानीकं व्यूढं दुर...,Sanjay said: On observing the Pandava army sta...
2,पश्यैतां पाण्डुपुत्राणामाचार्य महतीं चमूम् ।व्...,Duryodhan said: Respected teacher! Behold the ...
3,अत्र शूरा महेष्वासा भीमार्जुनसमा युधि,Behold in their ranks are many powerful warrio...
4,अत्र शूरा महेष्वासा भीमार्जुनसमा युधि,Behold in their ranks are many powerful warrio...


In [9]:
lines.eng=lines.eng.apply(lambda x: x.lower())
lines.hin=lines.hin.apply(lambda x: x.lower())

In [10]:
# Remove quotes
lines.eng=lines.eng.apply(lambda x: re.sub("'", '', x))
lines.hin=lines.hin.apply(lambda x: re.sub("'", '', x))
lines.shape

(700, 2)

In [11]:
exclude = set(string.punctuation) 
# Remove special characters
lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.hin=lines.hin.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))



In [12]:
# Remove numbers 
remove_digits = str.maketrans('', '', digits)
lines.eng=lines.eng.apply(lambda x: x.translate(remove_digits))
lines.hin = lines.hin.apply(lambda x: re.sub("[२३०८१५७९४६।]", "", x))


In [13]:
# Remove spaces
lines.eng=lines.eng.apply(lambda x: x.strip())
lines.hin=lines.hin.apply(lambda x: x.strip())
lines.eng=lines.eng.apply(lambda x: re.sub(" +", " ", x))
lines.hin=lines.hin.apply(lambda x: re.sub(" +", " ", x))

In [14]:

lines.hin = lines.hin.apply(lambda x : 'START_ '+ x + ' _END')

In [15]:
lines.sample(10)

,hin,eng
231,START_ भोक्तारं यज्ञतपसां सर्वलोकमहेश्वरम् सुह...,having realized me as the enjoyer of all sacri...
607,START_ देवद्विजगुरुप्राज्ञपूजनं शौचमार्जवम् ब्...,worship of the supreme lord the brahmins the s...
72,START_ जातस्य हि ध्रुवो मृत्युर्ध्रुवं जन्म मृ...,death is certain for one who has been born and...
284,START_ एतद्योनीनि भूतानि सर्वाणीत्युपधारय अहं ...,know that all living beings are manifested by ...
585,START_ अनेकचित्तविभ्रान्ता मोहजालसमावृता प्रसक...,possessed and led astray by such imaginings en...
459,START_ श्रीभगवानुवाच मया प्रसन्नेन तवार्जुनेदं...,the blessed lord said arjun being pleased with...
619,START_ सद्भावे साधुभावे च सदित्येतत्प्रयुज्यते...,the word “sat” means eternal existence and goo...
358,START_ अनन्याश्चिन्तयन्तो मां ये जना पर्युपासत...,there are those who always think of me and eng...
356,START_ त्रैविद्या मां सोमपा पूतपापायज्ञैरिष्ट्...,those who are inclined to the fruitive activit...
198,START_ न हि ज्ञानेन सदृशं पवित्रमिह विद्यते तत...,in this world there is nothing as purifying as...


In [16]:
#word English
all_eng_words=set()
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

#word hindi 
all_hindi_words=set()
for hin in lines.hin:
    for word in hin.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)

In [17]:
# Max Length
lenght_list=[]
for l in lines.eng:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)
max_length_src

139

In [18]:
# Max Length
lenght_list=[]
for l in lines.hin:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)
max_length_tar

42

In [19]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hindi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hindi_words)
num_encoder_tokens, num_decoder_tokens

(2861, 4046)

In [20]:
num_decoder_tokens += 1
num_encoder_tokens += 1
# For zero padding

In [21]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [22]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [23]:
lines = shuffle(lines)
lines.head(10)

,hin,eng
405,START_ बृहत्साम तथा साम्नां गायत्री छन्दसामहम्...,amongst the hymns in the sāma veda know me to ...
6,START_ अस्माकं तु विशिष्टा ये तान्निबोध द्विजो...,o best of brahmins hear too about the principa...
157,START_ इन्द्रियाणि मनो बुद्धिरस्याधिष्ठानमुच्य...,the senses mind and intellect are said to be b...
106,START_ तानि सर्वाणि संयम्य युक्त आसीत मत्पर वश...,they are established in perfect knowledge who ...
140,START_ यदि ह्यहं न वर्तेयं जातु कर्मण्यतन्द्रि...,for if i did not carefully perform the prescri...
348,START_ मोघाशा मोघकर्माणो मोघज्ञाना विचेतस राक्...,bewildered by the material energy such persons...
570,START_ श्रीभगवानुवाच अभयं सत्वसंशुद्धिर्ज्ञानय...,the supreme divine personality said o scion of...
598,START_ अशास्त्रविहितं घोरं तप्यन्ते ये तपो जना...,some people perform stern austerities that are...
431,START_ अनादिमध्यान्तमनन्तवीर्यमनन्तबाहुं शशिसू...,you are without beginning middle or end your p...
512,START_ ध्यानेनात्मनि पश्यन्ति केचिदात्मानमात्म...,some try to perceive the supreme soul within t...


In [24]:
# Train - Test Split
X, y = lines.eng, lines.hin
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
X_train.shape, X_test.shape

((630,), (70,))

In [25]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [26]:
latent_dim = 50

In [27]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [28]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)

# Use a softmax to generate a probability distribution over the target vocabulary for each time step
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [29]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 50)     143100      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 50)     202350      input_2[0][0]                    
____________________________________________________________________________________________

In [30]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 10

In [31]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

ValueError: ignored

In [ ]:
model.save_weights('nmt_weights.h5')

In [ ]:
model.load_weights('nmt_weights.h5')

In [ ]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
train_gen = generate_batch(X_test, y_test, batch_size = 1)
k=-1

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual hindi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted hindi Translation:', decoded_sentence[:-4])

Input English sentence: please describe to me your divine opulences by which you pervade all the worlds and reside in them o supreme master of yog how may i know you and think of you and while meditating in what forms can i think of you o supreme divine personality
Actual hindi Translation:  वक्तुमर्हस्यशेषेण दिव्या ह्यात्मविभूतय याभिर्विभूतिभिर्लोकानिमांस्त्वं व्याप्य तिष्ठसि कथं विद्यामहं योगिंस्त्वां सदा परिचिन्तयन् केषु केषु च भावेषु चिन्त्योऽसि भगवन्मया 
Predicted hindi Translation:  आर्जवम् आर्जवम् व्यवसायात्मिका योगस्त्वया योगस्त


In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual hindi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted hindi Translation:', decoded_sentence[:-4])

Input English sentence: that unintelligent resolve is said to be determination in the mode of ignorance in which one does not give up dreaming fearing grieving despair and conceit
Actual hindi Translation:  यया स्वप्नं भयं शोकं विषादं मदमेव च न विमुञ्चति दुर्मेधा धृति सा पार्थ तामसी 
Predicted hindi Translation:  सविज्ञानमिदं विदित्वायोगी मोह तत्समासेन तत्समासेन सा


In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual hindi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted hindi Translation:', decoded_sentence[:-4])

Input English sentence: i am ashamed of having been lazy
Actual hindi Translation:  मुझे अपने आलसी होने पर शर्म आती है 
Predicted hindi Translation:  मुझे दोनो पर पास डॉक्टर से आया 


In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual hindi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted hindi Translation:', decoded_sentence[:-4])

Input English sentence: its an absolute waste of time to wait any longer
Actual hindi Translation:  और इंतेज़ार करना समय की सरासर बर्बादी होगी 
Predicted hindi Translation:  खाना खाने के लिए और खाना खाने का खाना बनाना सीखना च


In [ ]:

k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual hindi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted hindi Translation:', decoded_sentence[:-4])

Input English sentence: ive quit drinking beer
Actual hindi Translation:  मैंने बीयर पीना छोड़ दिया है 
Predicted hindi Translation:  मैंने बीयर पीना छोड़ दिया है 


In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual hindi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted hindi Translation:', decoded_sentence[:-4])

Input English sentence: i have a dog
Actual hindi Translation:  मेरे पास एक कुत्ता है 
Predicted hindi Translation:  मेरा एक दोस्त है 


In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual hindi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted hindi Translation:', decoded_sentence[:-4])

Input English sentence: we hurried to the train station
Actual hindi Translation:  हम जल्दबाज़ी में ट्रेन स्टेशन गए 
Predicted hindi Translation:  हम जब में वाक्य ही चाहिए थी 


In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual hindi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted hindi Translation:', decoded_sentence[:-4])

Input English sentence: what time does your watch say it is now
Actual hindi Translation:  तुम्हारी घड़ी अब क्या समय बता रही है 
Predicted hindi Translation:  तुम्हारी घड़ी अब क्या समय बता रही है 


In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual hindi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted hindi Translation:', decoded_sentence[:-4])

Input English sentence: im sorry but i cant hear you well
Actual hindi Translation:  माफ़ कीजिएगा पर मैं आपको ठीक से सुन नहीं पा रहा हूँ 
Predicted hindi Translation:  माफ़ कीजिएगा पर मैं अपनी मदद से चिट्ठी खरीदने का पछ


In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual hindi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted hindi Translation:', decoded_sentence[:-4])

Input English sentence: are you free on tuesday
Actual hindi Translation:  तुम मंगलवार को खाली हो क्या 
Predicted hindi Translation:  तुम मंगलवार को खाली हो क्या 


In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual hindi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted hindi Translation:', decoded_sentence[:-4])

Input English sentence: strange as it may sound this is true
Actual hindi Translation:  चाहे सुनने में जितना भी अजीब लगे यह सच है 
Predicted hindi Translation:  एक अरब लोग सबसे कीमती अब अब रहा है 


In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual hindi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted hindi Translation:', decoded_sentence[:-4])

Input English sentence: he came back from america
Actual hindi Translation:  वह अमरीका से वापस आ गया 
Predicted hindi Translation:  वह एक पैसे जा रहा है 


In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual hindi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted hindi Translation:', decoded_sentence[:-4])

Input English sentence: why dont you come visit us
Actual hindi Translation:  तुम हमसे मिलने क्यों नहीं आते 
Predicted hindi Translation:  तुम किस नाप के बारे में करता है 
